<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/Fundus-eye-disease-detection/resnet50_2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install scikit-learn
!pip install google-colab

import numpy as np
import tensorflow as tf
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import cv2
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 224
CATEGORIES = ['cataract', 'diabetic_retinopathy', 'glaucoma', 'normal']
BATCH_SIZE = 32
EPOCHS = 25
DATA_DIR = '/content/drive/MyDrive/data set/dataset'
SPLIT_DIR = '/content/split_dataset'
PCA_COMPONENTS = 50

# Function to convert images to grayscale and apply PCA
def preprocess_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for category in CATEGORIES:
        category_path = os.path.join(input_dir, category)
        output_category_path = os.path.join(output_dir, category)

        if not os.path.exists(output_category_path):
            os.makedirs(output_category_path)

        images = os.listdir(category_path)
        processed_images = []

        for img in images:
            img_path = os.path.join(category_path, img)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            processed_images.append(image.flatten())

        processed_images = np.array(processed_images)
        pca = PCA(n_components=PCA_COMPONENTS)
        transformed_images = pca.fit_transform(processed_images)

        for i, img_data in enumerate(transformed_images):
            img_resized = cv2.resize(img_data.reshape(1, -1), (IMG_SIZE, IMG_SIZE))
            output_img_path = os.path.join(output_category_path, images[i])
            cv2.imwrite(output_img_path, img_resized)

# Apply preprocessing
preprocess_images(DATA_DIR, SPLIT_DIR)

# Create train and val directories
for dataset_type in ['train', 'val']:
    dataset_dir = os.path.join(SPLIT_DIR, dataset_type)
    if not os.path.exists(dataset_dir):
        os.makedirs(dataset_dir)
    for category in CATEGORIES:
        category_dir = os.path.join(dataset_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)

# Split data into train and validation sets
for category in CATEGORIES:
    source_dir = os.path.join(SPLIT_DIR, category)
    files = os.listdir(source_dir)
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42) # Split 80/20

    # Move files to train directory
    for file in train_files:
        shutil.move(os.path.join(source_dir, file), os.path.join(SPLIT_DIR, 'train', category, file))

    # Move files to validation directory
    for file in val_files:
        shutil.move(os.path.join(source_dir, file), os.path.join(SPLIT_DIR, 'val', category, file))


# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Changed color_mode to 'rgb'
train_generator = train_datagen.flow_from_directory(
    os.path.join(SPLIT_DIR, 'train'), target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', color_mode='rgb'
)
val_generator = val_datagen.flow_from_directory(
    os.path.join(SPLIT_DIR, 'val'), target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', color_mode='rgb'
)

# Compute class weights
y_train = train_generator.classes
class_weights_dict = dict(enumerate(compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train
)))

# Load pre-trained ResNet50
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Unfreeze last 15 layers for fine-tuning
for layer in resnet_base.layers[:-15]:
    layer.trainable = False

# Feature extraction
x = GlobalAveragePooling2D()(resnet_base.output)
x = Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = BatchNormalization()(x)  # Before activation
x = Activation('relu')(x)
x = Dropout(0.3)(x)  # Reduced dropout to 0.3
output = Dense(len(CATEGORIES), activation='softmax')(x)

# Model
model = Model(inputs=resnet_base.input, outputs=output)

# Compile with Adam optimizer and a lower initial learning rate
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower initial LR
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-6),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('/content/best_eye_disease_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weights_dict,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Evaluate the model
val_loss, val_acc = model.evaluate(val_generator, steps=len(val_generator))
print(f"Validation Accuracy: {val_acc:.4f}")

# Get predictions
y_pred_probs = model.predict(val_generator, steps=len(val_generator))
y_pred = np.argmax(y_pred_probs, axis=1)

from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(val_generator.classes, y_pred, target_names=CATEGORIES)
print(report)

# Calculate accuracy
accuracy = accuracy_score(val_generator.classes, y_pred)
print(f"Accuracy on Validation Set: {accuracy:.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.4 MB/s eta 0:00:00
Mounted at /content/drive
Found 3381 images belonging to 4 classes.
Found 847 images belonging to 4 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/25


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.3487 - loss: 2.0473
Epoch 1: val_loss improved from inf to 1.83472, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 50s 269ms/step - accuracy: 0.3491 - loss: 2.0461 - val_accuracy: 0.2739 - val_loss: 1.8347 - learning_rate: 1.0000e-04
Epoch 2/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.5225 - loss: 1.5606
Epoch 2: val_loss improved from 1.83472 to 1.71379, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 14s 128ms/step - accuracy: 0.5226 - loss: 1.5607 - val_accuracy: 0.3707 - val_loss: 1.7138 - learning_rate: 1.0000e-04
Epoch 3/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.6109 - loss: 1.3842
Epoch 3: val_loss improved from 1.71379 to 1.57934, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 126ms/step - accuracy: 0.6109 - loss: 1.3842 - val_accuracy: 0.5030 - val_loss: 1.5793 - learning_rate: 1.0000e-04
Epoch 4/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.7058 - loss: 1.1837
Epoch 4: val_loss improved from 1.57934 to 1.53069, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 124ms/step - accuracy: 0.7057 - loss: 1.1839 - val_accuracy: 0.5325 - val_loss: 1.5307 - learning_rate: 1.0000e-04
Epoch 5/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7812 - loss: 1.0132
Epoch 5: val_loss improved from 1.53069 to 1.48034, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 125ms/step - accuracy: 0.7809 - loss: 1.0135 - val_accuracy: 0.5596 - val_loss: 1.4803 - learning_rate: 1.0000e-04
Epoch 6/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.8310 - loss: 0.8951
Epoch 6: val_loss did not improve from 1.48034
106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 118ms/step - accuracy: 0.8310 - loss: 0.8952 - val_accuracy: 0.5667 - val_loss: 1.5356 - learning_rate: 1.0000e-04
Epoch 7/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.8810 - loss: 0.7794
Epoch 7: val_loss did not improve from 1.48034
106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 123ms/step - accuracy: 0.8809 - loss: 0.7796 - val_accuracy: 0.5939 - val_loss: 1.5576 - learning_rate: 1.0000e-04
Epoch 8/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.9019 - loss: 0.7103
Epoch 8: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.

Epoch 8: val_loss did not improve from 1.48034
106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 120ms/step - accuracy: 0.9020 - loss: 0.7102

106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 128ms/step - accuracy: 0.9447 - loss: 0.5956 - val_accuracy: 0.6375 - val_loss: 1.3776 - learning_rate: 1.0000e-05
Epoch 10/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.9770 - loss: 0.5384
Epoch 10: val_loss did not improve from 1.37763
106/106 ━━━━━━━━━━━━━━━━━━━━ 19s 119ms/step - accuracy: 0.9770 - loss: 0.5383 - val_accuracy: 0.6446 - val_loss: 1.3795 - learning_rate: 1.0000e-05
Epoch 11/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.9894 - loss: 0.5052
Epoch 11: val_loss improved from 1.37763 to 1.35925, saving model to /content/best_eye_disease_model.h5


106/106 ━━━━━━━━━━━━━━━━━━━━ 14s 128ms/step - accuracy: 0.9895 - loss: 0.5051 - val_accuracy: 0.6482 - val_loss: 1.3593 - learning_rate: 1.0000e-05
Epoch 12/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.9920 - loss: 0.4949
Epoch 12: val_loss did not improve from 1.35925
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 0.9920 - loss: 0.4949 - val_accuracy: 0.6505 - val_loss: 1.3684 - learning_rate: 1.0000e-05
Epoch 13/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.9956 - loss: 0.4768
Epoch 13: val_loss did not improve from 1.35925
106/106 ━━━━━━━━━━━━━━━━━━━━ 13s 119ms/step - accuracy: 0.9956 - loss: 0.4768 - val_accuracy: 0.6612 - val_loss: 1.3635 - learning_rate: 1.0000e-05
Epoch 14/25
106/106 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.9961 - loss: 0.4681
Epoch 14: ReduceLROnPlateau reducing learning rate to 1e-06.

Epoch 14: val_loss did not improve from 1.35925
106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 122ms/step - accuracy: 0.9961 - loss: 0.4681 - val_ac